In [1]:
%load_ext autoreload
%autoreload 1

In [2]:
%aimport utils, Networks

In [3]:
import numpy as np
import tensorflow as tf
import os
from os import path
from utils import load_embeddings_and_ids, concatenate_featmats
from sklearn.preprocessing import StandardScaler 
from Networks import ContentBasedLearn2RankNetwork_Precomputation

In [4]:
os.environ["CUDA_VISIBLE_DEVICES"]="0"

In [5]:
resnet50 = load_embeddings_and_ids('/mnt/workspace/Ugallery/ResNet50/', 'flatten_1.npy', 'ids')

In [6]:
resnext101 = load_embeddings_and_ids('/mnt/workspace/Ugallery/resnext101_32x8d_wsl/', 'features.npy', 'ids.npy')

In [6]:
# grammat_b1c1 = load_embeddings_and_ids('/mnt/workspace/Ugallery/gram_matrix/', 'block1_conv1.npy', 'ids')

In [7]:
# alexnet = load_embeddings_and_ids('/mnt/workspace/Ugallery/AlexNet/', 'fc7.npy', 'ids.npy')

In [8]:
# inceptionv3 = load_embeddings_and_ids('/mnt/workspace/Ugallery/InceptionV3/', 'avg_pool.npy', 'ids')

In [7]:
embedding_list = [
    resnet50,
    resnext101,
#     grammat_b1c1,
#     alexnet,
#     inceptionv3,
]

In [8]:
artwork_ids_set = set()
for embedding in embedding_list:
    if len(artwork_ids_set) == 0:        
        artwork_ids_set.update(embedding['index2id'])
    else:
        artwork_ids_set.intersection_update(embedding['index2id'])
artwork_ids = list(artwork_ids_set)
artwork_id2index = {_id:i for i,_id in enumerate(artwork_ids)}
n_artworks = len(artwork_ids)
n_artworks

13297

In [9]:
featmat_list = [tmp['featmat'] for tmp in embedding_list]
id2index_list = [tmp['id2index'] for tmp in embedding_list]
concat_featmat = concatenate_featmats(artwork_ids, featmat_list, id2index_list)

In [10]:
concat_featmat = StandardScaler().fit_transform(concat_featmat)

In [11]:
concat_featmat[0], concat_featmat[-1]

(array([-0.50733465, -0.77183085, -0.38606572, ...,  1.17075316,
        -0.61837645, -0.01425705]),
 array([ 1.2333275 ,  1.61747025, -0.48316753, ..., -0.61435814,
         2.32308702, -0.45504793]))

In [12]:
concat_featmat.shape

(13297, 4096)

In [15]:
!ls -a /mnt/workspace/pamessina_models/ugallery/youtube_like/v22\(10M-400K,avg+max,rsnt50,u\(300,300,200\)i\(200,200\),+p\(rl\&fk\)-np,fg.6,vcf.1,wd.0001\)

.  ..  checkpoint  .data-00000-of-00001  .index  .meta	train_logs.csv


In [16]:
!ls -a /home/mcartagena/Image-Embedding-Learning-and-Feature-Extraction/cold_start_experiments_without_users2trx_pablo/

.  ..  checkpoint  .data-00000-of-00001  .index  .meta	train_logs.csv


In [13]:
MODEL_PATH = '/mnt/workspace/pamessina_models/ugallery/youtube_like/v34(10M-500K,avg+max,rsnt50+rsnxt101,u(300,300,200)i(200,200),hyb(fa+dnn),rl(full),ab.2,cm.18,fg.7,wd.0001)/'
# MODEL_PATH = '/home/mcartagena/Image-Embedding-Learning-and-Feature-Extraction/'
MODEL_PATH

'/mnt/workspace/pamessina_models/ugallery/youtube_like/v34(10M-500K,avg+max,rsnt50+rsnxt101,u(300,300,200)i(200,200),hyb(fa+dnn),rl(full),ab.2,cm.18,fg.7,wd.0001)/'

In [14]:
with tf.Graph().as_default():
    network = ContentBasedLearn2RankNetwork_Precomputation(
        pretrained_embedding_dim=concat_featmat.shape[1],
        item_layer_units=[200,200],
    )
    with tf.Session() as sess:
        saver = tf.train.Saver()
        saver.restore(sess, tf.train.latest_checkpoint(MODEL_PATH))
        item_vectors = network.precompute_tensors(sess, concat_featmat)

INFO:tensorflow:Restoring parameters from /mnt/workspace/pamessina_models/ugallery/youtube_like/v34(10M-500K,avg+max,rsnt50+rsnxt101,u(300,300,200)i(200,200),hyb(fa+dnn),rl(full),ab.2,cm.18,fg.7,wd.0001)/


In [15]:
item_vectors.shape

(13297, 200)

In [16]:
dir_path = '/mnt/workspace/Ugallery/Youtube-like/v34(10M-500K,avg+max,rsnt50+rsnxt101,u(300,300,200)i(200,200),hyb(fa+dnn),rl(full),ab.2,cm.18,fg.7,wd.0001)/'
os.makedirs(dir_path, exist_ok=True)
item_vectors.dump(dir_path + "item_vectors.npy")
with open(dir_path + 'ids', 'w') as f:
    for _id in artwork_ids:
        f.write('%d\n' % _id)